In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/test_set/lung_aca/lungaca4231.jpeg
/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/test_set/lung_aca/lungaca4646.jpeg
/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/test_set/lung_aca/lungaca4483.jpeg
/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/test_set/lung_aca/lungaca4709.jpeg
/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/test_set/lung_aca/lungaca4457.jpeg
/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/test_set/lung_aca/lungaca4810.jpeg
/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/test_set/lung_aca/lungaca4993.jpeg
/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/test_set/lung_aca/lungaca4764.jpeg
/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/test_set/lung_aca/lungaca4839.jpeg
/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/test_set/lung_aca/lungaca4568.jpeg
/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/t

In [8]:
import os 
import tensorflow as tf 
from tensorflow.keras.layers import Dense,GlobalAvgPool2D,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50

In [3]:
train_datagen=ImageDataGenerator(
rescale=1/255,zoom_range=0.15,horizontal_flip=True,rotation_range=30,shear_range=0.15)

val_datagen=ImageDataGenerator(
rescale=1/255)

In [4]:
train_data=train_datagen.flow_from_directory(
directory='/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/training_set',
batch_size=32,
target_size=(224,224),
class_mode='categorical')

val_data=val_datagen.flow_from_directory(
directory='/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/test_set',
batch_size=32,
target_size=(224,224),
class_mode='categorical')

Found 12000 images belonging to 3 classes.
Found 2997 images belonging to 3 classes.


In [7]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the image
img_path = '/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/single_prediction/lung_cancer_type_1.jpeg'
img = load_img(img_path, target_size=(224, 224))

# Convert the image to an array
img_array = img_to_array(img)

# Check the shape of the image array
print("Image shape:", img_array.shape)


Image shape: (224, 224, 3)


In [9]:
base_model=ResNet50(input_shape=(224, 224, 3),weights='imagenet',include_top=False)

for layer in base_model.layers:
    layer.trainable=False

x=GlobalAveragePooling2D()(base_model.output)
x=Dense(1024,activation='relu')(x)
prediction_layer=Dense(3,activation='softmax')(x)

model = Model(inputs=base_model.input,outputs=prediction_layer)

model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()
    
    

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 25,688,963 (98.00 MB)

 Trainable params: 2,101,251 (8.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [10]:
history=model.fit(train_data,epochs=10,validation_data=val_data,batch_size=32,verbose=1)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1729324944.122533    1150 service.cc:145] XLA service 0x7970dc002290 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729324944.122606    1150 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1729324944.122612    1150 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  2/375 ━━━━━━━━━━━━━━━━━━━━ 31s 85ms/step - accuracy: 0.2422 - loss: 1.7210   

I0000 00:00:1729324950.193486    1150 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


375/375 ━━━━━━━━━━━━━━━━━━━━ 275s 685ms/step - accuracy: 0.4456 - loss: 1.1328 - val_accuracy: 0.5686 - val_loss: 0.8774
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 209s 547ms/step - accuracy: 0.6401 - loss: 0.8067 - val_accuracy: 0.6356 - val_loss: 0.7167
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 207s 543ms/step - accuracy: 0.6843 - loss: 0.6981 - val_accuracy: 0.7888 - val_loss: 0.5206
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 207s 543ms/step - accuracy: 0.7493 - loss: 0.5730 - val_accuracy: 0.7925 - val_loss: 0.4894
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 204s 535ms/step - accuracy: 0.7599 - loss: 0.5368 - val_accuracy: 0.7621 - val_loss: 0.5535
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 205s 537ms/step - accuracy: 0.7637 - loss: 0.5215 - val_accuracy: 0.7851 - val_loss: 0.4811
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 207s 543ms/step - accuracy: 0.7729 - loss: 0.5130 - val_accuracy: 0.8038 - val_loss: 0.4465
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 212s 554ms/step - accuracy: 0.7933 - loss: 0.47

In [11]:
for layer in base_model.layers:
    layer.trainable=True
    
model.compile(optimizer=Adam(learning_rate=0.00001),loss='categorical_crossentropy',metrics=['accuracy'])    
 
    

In [12]:
model.fit(train_data,epochs=10,validation_data=val_data,batch_size=32,verbose=1)

Epoch 1/10


I0000 00:00:1729327200.916097    1148 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_30', 20 bytes spill stores, 20 bytes spill loads



375/375 ━━━━━━━━━━━━━━━━━━━━ 278s 584ms/step - accuracy: 0.6896 - loss: 13.4928 - val_accuracy: 0.3333 - val_loss: 160.5016
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 214s 560ms/step - accuracy: 0.9328 - loss: 0.5568 - val_accuracy: 0.5262 - val_loss: 15.8336
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 222s 581ms/step - accuracy: 0.9534 - loss: 0.3085 - val_accuracy: 0.8599 - val_loss: 2.1832
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 214s 561ms/step - accuracy: 0.9653 - loss: 0.1641 - val_accuracy: 0.9793 - val_loss: 0.0904
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 214s 561ms/step - accuracy: 0.9765 - loss: 0.1239 - val_accuracy: 0.9813 - val_loss: 0.0696
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 223s 583ms/step - accuracy: 0.9786 - loss: 0.0823 - val_accuracy: 0.9857 - val_loss: 0.0477
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 219s 573ms/step - accuracy: 0.9826 - loss: 0.0674 - val_accuracy: 0.9857 - val_loss: 0.0502
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 217s 570ms/step - accuracy: 0.9857 - loss: 

In [13]:
model.evaluate(val_data)

94/94 ━━━━━━━━━━━━━━━━━━━━ 13s 136ms/step - accuracy: 0.9894 - loss: 0.0319


[0.028480224311351776, 0.9906573295593262]

In [14]:
model.save('lung_cancer_classification_model.h5')

In [19]:
from PIL import Image, ImageOps
import numpy as np

best_model=tf.keras.models.load_model('/kaggle/working/lung_cancer_classification_model.h5')

class_labels = ['Adenocarcinoma', 'Neuroendocrine tumors', 'Squamous cell carcinoma']

uploaded_file = '/kaggle/input/lung-cancer-images12000-imagesmostly/dataset/single_prediction/lung_cancer_type_3.jpeg'

# Preprocessing function to handle image
from PIL import ImageOps, Image
import numpy as np

def preprocess_image(image_data):
    """
    Preprocess the uploaded image to fit the model input
    """
    image = Image.open(image_data)
    # Use LANCZOS for high-quality downsampling
    image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)  # Resize to model input size
    img = np.asarray(image)
    img = img.astype(np.float32) / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img


img = preprocess_image(uploaded_file)
prediction = best_model.predict(img)
predicted_class = np.argmax(prediction, axis=1)
confidence = prediction[0]
adenocarcinoma_confidence = confidence[0] * 100
neuroendocrine_confidence = confidence[1] * 100
squamous_confidence = confidence[2] * 100
    
    # Display results
print(f"### Predicted Class: *{class_labels[predicted_class[0]]}*")

print(f"#### Confidence Levels:")
print(f"- *Adenocarcinoma*: {adenocarcinoma_confidence:.2f}%")
print(f"- *Neuroendocrine Tumors*: {neuroendocrine_confidence:.2f}%")
print(f"- *Squamous Cell Carcinoma*: {squamous_confidence:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
### Predicted Class: *Neuroendocrine tumors*
#### Confidence Levels:
- *Adenocarcinoma*: 0.00%
- *Neuroendocrine Tumors*: 100.00%
- *Squamous Cell Carcinoma*: 0.00%


In [21]:
!pip install streamlit

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 55.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.3 MB/s eta 0:00:00:00:0100:01


In [22]:
import streamlit as st
import tensorflow as tf
from PIL import Image, ImageOps
import numpy as np

# Load the pre-trained model for lung cancer classification
@st.cache(allow_output_mutation=True)
def load_model():
    model = tf.keras.models.load_model('/kaggle/working/lung_cancer_classification_model.h5')  # Assuming you have trained the model for 3 classes
    return model

model = load_model()

# Streamlit title and description
st.title("Lung Cancer Classification")
st.write("""
This application classifies lung cancer into three categories: 
1. Adenocarcinoma 
2. Neuroendocrine tumors 
3. Squamous cell carcinoma
Upload a lung CT scan or X-ray image to get a prediction.
""")

# Sidebar for patient information
st.sidebar.title("Patient Information")
name = st.sidebar.text_input("Patient Name")
age = st.sidebar.number_input("Age", min_value=0, max_value=120, value=30)
gender = st.sidebar.selectbox("Gender", ["Male", "Female", "Other"])

st.sidebar.write("Please enter correct patient details before proceeding.")

# File uploader for image input
uploaded_file = st.file_uploader("Choose an image (CT scan or X-ray)...", type=["jpg", "jpeg", "png"])

def preprocess_image(image_data):
    """
    Preprocess the uploaded image to fit the model input
    """
    image = Image.open(image_data)
    # Use LANCZOS for high-quality downsampling
    image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)  # Resize to model input size
    img = np.asarray(image)
    img = img.astype(np.float32) / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Class labels for lung cancer types
class_labels = ['Adenocarcinoma', 'Neuroendocrine tumors', 'Squamous cell carcinoma']

if uploaded_file is not None:
    # Display the uploaded image
    st.image(uploaded_file, caption="Uploaded Lung Image", use_column_width=True)
    st.write("Classifying the image...")

    # Preprocess the image and make predictions
    img = preprocess_image(uploaded_file)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction, axis=1)  # Get the index of the highest probability
    
    # Get prediction confidence for each class
    confidence = prediction[0]
    adenocarcinoma_confidence = confidence[0] * 100
    neuroendocrine_confidence = confidence[1] * 100
    squamous_confidence = confidence[2] * 100
    
    # Display results
    st.write(f"### Predicted Class: *{class_labels[predicted_class[0]]}*")
    
    # Show the confidence scores for each class
    st.write(f"#### Confidence Levels:")
    st.write(f"- *Adenocarcinoma*: {adenocarcinoma_confidence:.2f}%")
    st.write(f"- *Neuroendocrine Tumors*: {neuroendocrine_confidence:.2f}%")
    st.write(f"- *Squamous Cell Carcinoma*: {squamous_confidence:.2f}%")
    
    st.write(f"Patient: {name}, Age: {age}, Gender: {gender}")
    st.write("*Note*: Please consult a medical professional for an official diagnosis.")

# Footer
st.write("*Disclaimer:* This tool is for educational purposes and is not intended for medical diagnosis.")

2024-10-19 09:22:45.637 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 09:22:45.751 
  command:

    streamlit run /opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-10-19 09:22:45.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 09:22:45.752 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.

2024-10-19 09:22:45.754 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-1

In [25]:
from IPython.display import FileLink

In [26]:
FileLink(r'/kaggle/working/lung_cancer_classification_model.h5')

/kaggle/working/lung_cancer_classification_model.h5

In [27]:
!pip show tensorflow


Name: tensorflow
Version: 2.16.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: explainable-ai-sdk, tensorflow-cloud, tensorflow-serving-api, tensorflow-text, tensorflow_decision_forests, tf_keras, witwidget


In [28]:
import tensorflow as tf
print(tf.__version__)


2.16.1
